# Data Cleaning Pipeline

This notebook cleans and prepares the PWT 11.0 dataset for analysis by handling data types, removing invalid entries, and encoding quality indicators.

In [17]:
import pandas as pd
import numpy as np

## Load Raw Data

In [18]:
df = pd.read_excel('dataset/pwt110.xlsx', sheet_name='Data')
print(f"Loaded {df.shape[0]:,} rows, {df.shape[1]} columns")

Loaded 13,693 rows, 51 columns


## Set Data Types

Convert categorical text columns and numerical columns to appropriate types for efficient processing.

In [19]:
text_cols = ['countrycode', 'country', 'currency_unit']
data_information_flags = ['i_cig', 'i_xm', 'i_xr', 'i_outlier', 'i_irr']

print(df.info())

for c in text_cols:
    df[c] = df[c].astype('category')

numerical_cols = [c for c in df.columns 
            if c not in text_cols + ['year'] + data_information_flags]
for c in numerical_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

print(df.info())
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13693 entries, 0 to 13692
Data columns (total 51 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   countrycode    13693 non-null  object 
 1   country        13693 non-null  object 
 2   currency_unit  13693 non-null  object 
 3   year           13693 non-null  int64  
 4   rgdpe          11204 non-null  float64
 5   rgdpo          11204 non-null  float64
 6   pop            11204 non-null  float64
 7   emp            10310 non-null  float64
 8   avh            5018 non-null   float64
 9   hc             9220 non-null   float64
 10  ccon           11204 non-null  float64
 11  cda            11204 non-null  float64
 12  cgdpe          11204 non-null  float64
 13  cgdpo          11204 non-null  float64
 14  cn             11031 non-null  float64
 15  ck             7750 non-null   float64
 16  ctfp           6991 non-null   float64
 17  cwtfp          6991 non-null   float64
 18  rgdpna

## Remove Invalid Entries

Drop rows with missing GDP (required for analysis) and remove duplicate country-year observations.

In [ ]:
df = df.dropna(subset=['rgdpo'])
df = df.drop_duplicates(subset=['countrycode', 'year'])
print(df.duplicated(subset=['countrycode', 'year']).sum())
print(df.isna().sum())

0


## Encode Data Quality Flags

Convert PWT's text-based quality indicators to numeric codes for easier filtering and analysis:

**i_cig_encoded**: Consumption/Investment/Government data quality (0=Extrapolated, 1=Benchmark, 2=Interpolated, 3-4=ICP PPP timeseries)

**i_xm_encoded**: Export/Import data quality (0=Extrapolated, 1=Benchmark, 2=Interpolated)

**i_xr_encoded**: Exchange rate source (0=Market-based, 1=Estimated)

**i_outlier_encoded**: Identifies outlier observations (0=Regular, 1=Outlier)

**i_irr_encoded**: Internal rate of return quality (0=Regular, 1=Low capital share, 2=Lower bound, 3=Outlier)

In [21]:
for col in ['i_cig', 'i_xm', 'i_xr', 'i_outlier', 'i_irr']:
    df[col] = df[col].astype(str).str.strip()

cig_map = {
    'Extrapolated': 0,
    'Benchmark': 1,
    'Interpolated': 2,
    'ICP PPP timeseries: benchmark or interpolated': 3,
    'ICP PPP timeseries: extrapolated': 4
}
xm_map = {'Extrapolated': 0, 'Benchmark': 1, 'Interpolated': 2}
irr_map = {
    'Regular': 0,
    'Low capital share': 1,
    'Lower bound': 2,
    'Outlier': 3
}

xr_map = {'Market-based': 0, 'Estimated': 1}
outlier_map = {'Regular': 0, 'Outlier': 1}


df['i_cig_encoded'] = df['i_cig'].map(cig_map).astype('Int8')
df['i_xm_encoded'] = df['i_xm'].map(xm_map).astype('Int8')
df['i_xr_encoded'] = df['i_xr'].map(xr_map).astype('Int8')
df['i_outlier_encoded'] = df['i_outlier'].map(outlier_map).astype('Int8')
df['i_irr_encoded'] = df['i_irr'].map(irr_map).astype('Int8')

for col in ['i_cig_encoded', 'i_xm_encoded', 'i_xr_encoded', 'i_outlier_encoded', 'i_irr_encoded']:
    n_missing = df[col].isna().sum()
    if n_missing:
        print(f"{col}: {n_missing} unmapped values")

df = df.drop(columns=['i_cig', 'i_xm', 'i_xr', 'i_outlier', 'i_irr'])



i_irr_encoded: 3000 unmapped values


## Data Quality Validation

Comprehensive checks for data integrity, outliers, missing values, and temporal coverage.

In [22]:
print("=== BASIC INFO ===")
print(f"Rows: {df.shape[0]:,} | Columns: {df.shape[1]}")
print(f"Countries: {df['countrycode'].nunique()} | Years: {df['year'].min()}–{df['year'].max()}")
print()

print("=== VALUE SANITY CHECKS ===")
invalid_pop = df.loc[df['pop'] <= 0, ['countrycode', 'year', 'pop']]
invalid_emp = df.loc[df['emp'] < 0, ['countrycode', 'year', 'emp']]
invalid_gdp = df.loc[df['rgdpo'] < 0, ['countrycode', 'year', 'rgdpo']]

print(f"Population <= 0: {len(invalid_pop)}")
print(f"Employment < 0: {len(invalid_emp)}")
print(f"Real GDP (rgdpo) < 0: {len(invalid_gdp)}")
print()

print("=== CROSS-VARIABLE CONSISTENCY ===")
inconsistent_emp = df.loc[df['emp'] > df['pop'], ['countrycode', 'year', 'emp', 'pop']]
print(f"Employment > Population: {len(inconsistent_emp)}")
print()

print("=== NUMERIC OUTLIER SCAN (|z| > 5) ===")
numeric_cols = df.select_dtypes(include=[np.number]).columns
z_scores = np.abs((df[numeric_cols] - df[numeric_cols].mean()) / df[numeric_cols].std())
outlier_counts = (z_scores > 5).sum().sort_values(ascending=False)
print(outlier_counts[outlier_counts > 0].head(10))
print()

print("=== MISSING DATA SUMMARY ===")
missing_summary = df.isna().mean().sort_values(ascending=False)
print(missing_summary.head(10))
print(f"\nOverall missingness: {df.isna().mean().mean():.2%}")
print()

print("=== TIME COVERAGE (first 5 countries) ===")
coverage = (
    df.groupby('countrycode', observed=True)['year']
      .agg(['min', 'max', 'count'])
      .sort_values('count', ascending=False)
)

print(coverage.head(5))
print()

print("=== DUPLICATE CHECK ===")
duplicates = df.duplicated(subset=['countrycode', 'year']).sum()
print(f"Duplicate country-year entries: {duplicates}")
print()

print("=== SUMMARY STATS (key columns) ===")
key_cols = ['rgdpo', 'pop', 'emp', 'labsh', 'irr']
for col in key_cols:
    if col in df.columns:
        desc = df[col].describe(percentiles=[.01, .5, .99])
        print(f"\n{col}:\n{desc.to_string()}")


=== BASIC INFO ===
Rows: 11,201 | Columns: 51
Countries: 185 | Years: 1950–2023

=== VALUE SANITY CHECKS ===
Population <= 0: 0
Employment < 0: 0
Real GDP (rgdpo) < 0: 0

=== CROSS-VARIABLE CONSISTENCY ===
Employment > Population: 0

=== NUMERIC OUTLIER SCAN (|z| > 5) ===
i_outlier_encoded    294
pop                  115
i_irr_encoded         98
emp                   95
rnna                  88
ck                    85
cgdpo                 74
ccon                  74
rgdpo                 74
cda                   73
dtype: Int64

=== MISSING DATA SUMMARY ===
cor_exp    0.811445
avh        0.552272
cwtfp      0.376127
rwtfpna    0.376127
rtfpna     0.376127
ctfp       0.376127
ck         0.308365
rkna       0.308365
pl_k       0.308365
irr        0.267833
dtype: float64

Overall missingness: 9.64%

=== TIME COVERAGE (first 5 countries) ===
              min   max  count
countrycode                   
AUS          1950  2023     74
ARG          1950  2023     74
BRA          1950  2023 

# Export Data

In [23]:
df.to_csv('dataset/pwt110_cleaned.csv', index=False)